In [10]:
import pandas as pd
import numpy as np

In [11]:
cleaned = pd.read_csv('C_cleaned2.csv')
# cleaned

In [12]:
predictors = cleaned[['Q5', 'Q76_6', 'Q69.1', 'Q69']].copy()

# Rename 
predictors = predictors.rename(columns= {'Q5': 'r3', 'Q76_6': 'r15', 'Q69.1': 'r28', 'Q69': 'd5'})
predictors

,r3,r15,r28,d5
0,1.0,6.0,1.0,3
1,36.0,6.0,1.0,3
2,28.0,4.0,2.0,3
3,24.0,6.0,3.0,3
4,5.0,6.0,3.0,2
...,...,...,...,...
346,3.0,4.0,4.0,3
347,28.0,NaN,NaN,3
348,10.0,6.0,1.0,2
349,6.0,NaN,NaN,3


In [13]:
# Calculate the mean for r12 variables
r12_variables = ['Q12', 'Q11', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17']
reverse_code_r12 = ['Q14', 'Q17']

for var in reverse_code_r12:
    cleaned[var] = 6 - cleaned[var]

cleaned['r12_mean'] = cleaned[r12_variables].mean(axis=1)


# Calculate the mean for r16 variables
r16_variables = ['Q19_1', 'Q19_2', 'Q19_3', 'Q19_4', 'Q19_5', 'Q19_6', 'Q19_7', 'Q19_8', 'Q19_9', 'Q19_10', 'Q19_11']
reverse_code_r16 = ['Q19_2', 'Q19_7']

for var in reverse_code_r16:
    cleaned[var] = 8 - cleaned[var]

cleaned['r16_mean'] = cleaned[r16_variables].mean(axis=1)

# Calculate the mean for r18 variables
r18_variables = ['Q18_1', 'Q18_2', 'Q18_3', 'Q18_4', 'Q18_5', 'Q18_6']
reverse_code_r18 = ['Q18_1', 'Q18_3', 'Q18_5', 'Q18_6']

for var in reverse_code_r18:
    cleaned[var] = 8 - cleaned[var]

cleaned['r18_mean'] = cleaned[r18_variables].mean(axis=1)


In [14]:
predictors['r12'] = cleaned['r12_mean']
predictors['r16'] = cleaned['r16_mean']
predictors['r18'] = cleaned['r18_mean']

In [15]:
# Create a temporary DataFrame with r17 columns
r17_df = cleaned[['Q65_1', 'Q65_2', 'Q65_3', 'Q65_4', 'Q65_5', 'Q65_6', 'Q65_7', 
                  'Q65_8', 'Q65_9', 'Q65_10', 'Q65_11', 'Q65_12', 'Q65_13']].copy()
    
# Define the custom transformation function for normal items
def normal_transformation(value):
    if value == 1.0:
        return 1
    elif value == 2.0:
        return -1
    elif value in [3.0, 4.0]:
        return 0.5
    else:
        return np.nan

    # Define the custom transformation function for reverse coding items
def reverse_transformation(value):
    if value == 1.0:
        return -1
    elif value == 2.0:
        return 1
    elif value in [3.0, 4.0]:
        return -0.5
    else:
        return np.nan
    
# Define columns for normal and reverse coding
normal_code_columns = ['Q65_1', 'Q65_2', 'Q65_3', 'Q65_4', 'Q65_5', 'Q65_6', 'Q65_7', 'Q65_11', 'Q65_12']
reverse_code_columns = ['Q65_8', 'Q65_9', 'Q65_10', 'Q65_13']

# Apply the custom transformation to all r17 columns
for col in normal_code_columns:
    r17_df[col] = r17_df[col].apply(normal_transformation)

for col in reverse_code_columns:
    r17_df[col] = r17_df[col].apply(reverse_transformation)
    
# Custom function to set r17 to NaN if all 13 items are NaN
def set_nan_if_all_nan(row):
    if row.isna().all():
        return np.nan
    else:
        return row.sum()

# Apply the custom function to calculate r17
predictors['r17'] = r17_df.apply(set_nan_if_all_nan, axis=1)
predictors

,r3,r15,r28,d5,r12,r16,r18,r17
0,1.0,6.0,1.0,3,3.857143,5.363636,4.166667,8.0
1,36.0,6.0,1.0,3,4.000000,5.454545,5.666667,12.5
2,28.0,4.0,2.0,3,3.000000,4.000000,4.000000,3.0
3,24.0,6.0,3.0,3,3.285714,4.272727,4.333333,6.0
4,5.0,6.0,3.0,2,2.571429,4.818182,3.500000,-1.0
...,...,...,...,...,...,...,...,...
346,3.0,4.0,4.0,3,2.714286,3.000000,3.166667,-3.0
347,28.0,NaN,NaN,3,NaN,NaN,NaN,NaN
348,10.0,6.0,1.0,2,4.571429,4.818182,5.666667,9.0
349,6.0,NaN,NaN,3,NaN,NaN,NaN,NaN


In [18]:
def create_f26(row):
    values = row[['Q58#2_1', 'Q58#2_2', 'Q58#2_3', 'Q58#2_4', 'Q58#2_5', 'Q58#2_6']].values
    
    if pd.isnull(values).all():
        return np.nan
    
    if 1 in values[5:]: # Check if Q58#2_6 or Q58#2_7 is selected
        return 1
    elif 1 in values[:5]:  # Check if Q58#2_1, Q58#2_2, Q58#2_3, Q58#2_4, or Q58#2_5 is selected
        return 0
    else:
        return np.nan

# Apply the function to the cleaned_data DataFrame to create the f26 variable
cleaned['f26'] = cleaned.apply(create_f26, axis=1)
predictors['f26'] = cleaned['f26']
predictors
# Add f26 to the final DataFrame
# final['f26'] = cleaned_data['f26']

,r3,r15,r28,d5,r12,r16,r18,r17,f26
0,1.0,6.0,1.0,3,3.857143,5.363636,4.166667,8.0,0.0
1,36.0,6.0,1.0,3,4.000000,5.454545,5.666667,12.5,0.0
2,28.0,4.0,2.0,3,3.000000,4.000000,4.000000,3.0,0.0
3,24.0,6.0,3.0,3,3.285714,4.272727,4.333333,6.0,0.0
4,5.0,6.0,3.0,2,2.571429,4.818182,3.500000,-1.0,NaN
...,...,...,...,...,...,...,...,...,...
346,3.0,4.0,4.0,3,2.714286,3.000000,3.166667,-3.0,0.0
347,28.0,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN
348,10.0,6.0,1.0,2,4.571429,4.818182,5.666667,9.0,NaN
349,6.0,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN


In [19]:
# Create the outcome variable 'r23' based on the 'Q68' variable in cleaned_data
predictors['r23'] = cleaned['Q68'].apply(lambda x: 1 if x == 1 else 0 if x in [2, 3, 4] else np.nan)

# Remove cases where 'f24' has no values
predictors = predictors.dropna(subset=['r23'])
predictors

,r3,r15,r28,d5,r12,r16,r18,r17,f26,r23
0,1.0,6.0,1.0,3,3.857143,5.363636,4.166667,8.0,0.0,1.0
1,36.0,6.0,1.0,3,4.000000,5.454545,5.666667,12.5,0.0,1.0
2,28.0,4.0,2.0,3,3.000000,4.000000,4.000000,3.0,0.0,1.0
3,24.0,6.0,3.0,3,3.285714,4.272727,4.333333,6.0,0.0,1.0
4,5.0,6.0,3.0,2,2.571429,4.818182,3.500000,-1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
338,24.0,4.0,3.0,3,3.571429,3.181818,4.166667,-3.5,NaN,0.0
339,12.0,6.0,1.0,4,4.428571,5.545455,4.666667,9.5,0.0,0.0
340,10.0,7.0,1.0,3,4.142857,5.636364,5.500000,12.5,0.0,1.0
346,3.0,4.0,4.0,3,2.714286,3.000000,3.166667,-3.0,0.0,0.0


In [21]:
r28_dummies = pd.get_dummies(predictors['r28'], prefix='r28', drop_first=False)
r28_dummies = r28_dummies.iloc[:, :-1]
r28_dummies.columns = [f'r28_{int(float(col.split("_")[-1]))}' for col in r28_dummies.columns]
predictors2 = pd.concat([predictors, r28_dummies], axis=1)

In [20]:
predictors.to_csv('C_predictors_no_manual.csv')

In [23]:
# Fill the missing values with the mean
predictors2 = predictors2.fillna(predictors2.mean())
predictors2.to_csv('C_predictors_no_manual2.csv')